In [51]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")

MODELS = {
    "gpt": "gpt-4o",
    "claude": "claude-3-5-sonnet-202401022",
    "mixtral": "mixtral:8x7b",
    "llama2": "llama2",
    "llama3": "llama3.3"
}

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_ollama.llms import OllamaLLM
from langchain_anthropic import ChatAnthropic

exec_model = "llama2"
if exec_model == "gpt":
    model = ChatOpenAI(api_key=OPENAI_API_KEY, model=MODELS["gpt"])
elif exec_model == "anthropic":
    model = ChatAnthropic(api_key=ANTHROPIC_API_KEY, model=MODELS["anthropic"])
else:
    model = OllamaLLM(model=MODELS["llama2"])

model.invoke("Tell me a joke")

"\nWhy don't scientists trust atoms? Because they make up everything! 😂"

In [53]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser

print(chain.invoke("Tell me a joke"))



Sure! Here's a classic one:

Why don't scientists trust atoms?
Because they make up everything!

I hope that brought a smile to your face!


In [24]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("feynman.pdf")

pages = loader.load_and_split()
pages



[Document(metadata={'producer': 'Skia/PDF m134', 'creator': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36', 'creationdate': '2025-03-29T18:58:41+00:00', 'title': 'The Feynman Lectures on Physics Vol. I Ch. 6: Probability', 'moddate': '2025-03-29T18:58:41+00:00', 'source': 'feynman.pdf', 'total_pages': 13, 'page': 0, 'page_label': '1'}, page_content='◄ ▲ ► \nA A A\n#6 Probability (10/13/61)\nThe recording of this lecture is missing\nfrom the Caltech Archives.\n6\nProbability\n(There was no summary for this lecture.)\n(There was no summary for this lecture.)\n“The true logic of this world is in the calculus of probabilities.”   —James\xa0Clerk\xa0Maxwell\n6–1Chance and likelihood\n“Chance” is a word which is in common use in everyday living. The radio reportsspeaking of tomorrow’s weather may say: “There is a sixty percent chance of rain.” Youmight say: “There is a small chance that I shall live to be one hundred ye

#### Need to create a prompt template to ask the model questions from a specific context

In [16]:
from langchain.prompts import PromptTemplate

template = """
Answer the question based on the context below. If you don't know the answer, just reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
print(prompt.format(context="Here is some context", question="Here is a question"))


Answer the question based on the context below. If you don't know the answer, just reply "I don't know".

Context: Here is some context

Question: Here is a question



Need to pass the prompt into the model via chaining

In [18]:
chain = prompt | model | parser

Need to pass in the context and question in order to invoke

In [19]:
chain.invoke(
    {
        "context": "The name I was given was Brandon",
        "question": "What is my name?"
    }
)

' Sure! Based on the context you provided, your name is Brandon.'

If we need to know how to use the prompt we've created, we can use `input_schema.schema`

In [22]:
chain.input_schema.schema()

/var/folders/fh/pr28tpnx5v58ycdrr7lrcv1h0000gn/T/ipykernel_4483/3226659032.py:1: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  chain.input_schema.schema()


{'properties': {'context': {'title': 'Context', 'type': 'string'},
  'question': {'title': 'Question', 'type': 'string'}},
 'required': ['context', 'question'],
 'title': 'PromptInput',
 'type': 'object'}

We want to pass only the relevant parts of a pdf document as a context

VectorStore to the rescue!

1. Serves as a database for the content by generating embeddings for each page of the pdf so we can compare the embeddings to the question that the user is asking

`uv add docarray`
pydantic should already be installed

In [54]:
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_openai import OpenAIEmbeddings
from langchain_ollama.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings(model=MODELS["llama2"]) # Since we're using Ollama, we need to use the OllamaEmbeddings class
# Create a vector store in memory, ideally we would use something like pinecone or supabase
vectorstore = DocArrayInMemorySearch.from_documents(
    pages,
    embedding=embeddings # Every model has its own way of creating embeddings
)


Turning the `vectorstore` into a `retriever` which is a component of langchain which will allow you to retrieve information from anywhere

In [55]:
retriever = vectorstore.as_retriever()
retriever.invoke("What did Feynman say about the second law of thermodynamics?")

[Document(metadata={'producer': 'Skia/PDF m134', 'creator': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36', 'creationdate': '2025-03-29T18:58:41+00:00', 'title': 'The Feynman Lectures on Physics Vol. I Ch. 6: Probability', 'moddate': '2025-03-29T18:58:41+00:00', 'source': 'feynman.pdf', 'total_pages': 13, 'page': 2, 'page_label': '3'}, page_content='We can generalize our reasoning to any situation in which there are \xa0different but“equivalent” (that is, equally likely) possible results of an observation. If an observationcan yield \xa0different results, and we have reason to believe that any one of them is aslikely as any other, then the probability of a particular outcome  is\xa0 .\nIf there are seven different-colored balls in an opaque box and we pick one out “atrandom” (that is, without looking), the probability of getting a ball of a particular color is. The probability that a “blind draw” from a shufﬂed de

Back to our chain. We need to provide a context and a question. 
The question is going to come from the retriever. 

The prompt is expecting a map.

In [46]:
from operator import itemgetter
chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
| prompt | model | parser )

chain.invoke({"question": "What's my name?"})


'Your name is Feynman.'

The retriever requires a question
`itemgetter` you can call a function with a dictionary and you get what you need which is a runnable
Piping the question into a retriever

In [56]:
chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question")
    } 
    | prompt
    | model 
    | parser
)
chain.invoke({"question": "What did Feynman say about probability?"})

'In Chapter 6 of "The Feynman Lectures on Physics," Feynman discusses probability and its role in describing the behavior of particles at the quantum level. He presents several key points regarding probability in physics:\n\n1. Probability is a fundamental aspect of quantum mechanics, and it cannot be avoided or wished away.\n2. The product of the widths of position and momentum probabilities must be greater than or equal to the number , which is the mass of the particle. This is known as the Heisenberg uncertainty principle.\n3. In attempting to describe the world in a different way, some physicists still have an intuitive conviction that it is possible to remove all this uncertainty about the way things are. However, no one has been successful in doing so.\n4. The necessary uncertainty in our specification of the position of a particle becomes most important when we wish to describe the structure of atoms. Specifically, there is a small probability of finding an electron at distances

In [57]:
question = "What did Feynman say about probability?"
print(f"Question: {question}")
print(f"Answer: {chain.invoke({'question': question})}")
print()

Question: What did Feynman say about probability?
Answer: Feynman said that in quantum mechanics, probabilities are the best way to describe nature, and that the uncertainty principle is a fundamental aspect of nature, stating that the product of the uncertainties in position and momentum is greater than or equal to h/2m, where h is Planck's constant and m is the mass of the particle. He also mentioned that particles behave in a funny way and that there are still some physicists who are working on removing this uncertainty, but no one has been successful yet.



In [58]:
for s in chain.stream({"question": question}):
    print(s, end="", flush=True)


In chapter 6 of The Feynman Lectures on Physics, Richard Feynman discusses the concept of probability in physics. He explains that probability is a fundamental aspect of quantum mechanics and cannot be avoided. Feynman highlights several key points related to probability:

1. Probability densities: Feynman shows how probability densities can be used to describe the position and velocity of particles. He uses figures to illustrate the shape of these probabilities.
2. Uncertainty principle: Feynman explains that there is a fundamental uncertainty in our specification of the position of a particle, which becomes particularly important when describing the structure of atoms. He notes that the number represents the "typical" radius of the atom.
3. Independence of widths: Feynman highlights that if we try to force particles to have a precise velocity or be in a particular place, nature demands that there must be some uncertainty involved. This means that we cannot choose independently how na

In [60]:
chain.batch([{"question": "What did Feynman say about probability?"}, {"question": "What did Feynman say about the second law of thermodynamics?"}])

['Feynman discussed the concept of probability in Chapter 6 of his "Feynman Lectures on Physics." He explained that probability is a fundamental aspect of quantum mechanics, and that particles behave in a way that is inherently uncertain.\n\nFeynman noted that the position and velocity of a particle cannot be precisely known at the same time, due to the uncertainty principle. He also discussed how the necessary uncertainty in our specification of the position of a particle becomes most important when describing the structure of atoms.\n\nFeynman stated that particles behave in a "funny way" and that there is an inherent fuzziness in any attempt to describe nature. He also noted that some physicists still work on the problem of removing this uncertainty, but no one has been successful so far.\n\nOverall, Feynman emphasized the importance of probability in quantum mechanics and the limitations of precision in describing the behavior of particles at the atomic level.',
 'In his lecture on